# Ollama Introduction
This little notebook is a brief introduction to Ollama, a tool for interacting with open LLMs deployed on Ollama server (the server can also run locally).

In [1]:
from ollama import Client

In [2]:
# initialize the Ollama client with the specified host
ollama_host = "http://localhost:11434"
client = Client(host=ollama_host)

In [6]:
# Get a list of all models that are currently downloaded
models = client.list()
print(models.models)

MODEL_NAME = "gemma3"

[Model(model='gemma3:latest', modified_at=datetime.datetime(2025, 6, 11, 16, 51, 6, 220107, tzinfo=TzInfo(+02:00)), digest='a2af6cc3eb7fa8be8504abaf9b04e88f17a119ec3f04a3addf55f92841195f5a', size=3338801804, details=ModelDetails(parent_model='', format='gguf', family='gemma3', families=['gemma3'], parameter_size='4.3B', quantization_level='Q4_K_M')), Model(model='gemma3:27b', modified_at=datetime.datetime(2025, 6, 11, 16, 45, 59, 91280, tzinfo=TzInfo(+02:00)), digest='a418f5838eaf7fe2cfe0a3046c8384b68ba43a4435542c942f9db00a5f342203', size=17396936941, details=ModelDetails(parent_model='', format='gguf', family='gemma3', families=['gemma3'], parameter_size='27.4B', quantization_level='Q4_K_M'))]


In [ ]:
# Send a simple prompt to the model
# model: select a model from the list of models obtained from client.list()
# messages: a list of messages containing the conversation history. Some models also 
# have a system message, to add this, make the first message:
# {"role": "system", "content": "Your system message here"}
# Gemma3 does not have a system message, so we can start with the user message.
# To add responses from the model, you can use the "assistant" role, i.e.:
# {"role": "assistant", "content": "The capital of France is Paris."}

response = client.chat(
    model=MODEL_NAME,
    messages=[
        {"role": "user", "content": "What is the capital of France?"},
        {"role": "assistant", "content": "The capital of France is Paris."},
        {"role": "user", "content": "And who many people live there?"},
    ],
)
print(response.message.content)

As of 2023, the population of Paris is approximately **2.1 million** people.

It’s important to note that this refers to the city proper (the administrative limits). The Greater Paris metropolitan area, which includes surrounding suburbs, has a much larger population – over 11 million! 

Would you like to know more about the population of Paris or its surrounding area?


In [7]:
# To control the decoding parameters, such as temperature, maxium number of tokens, etc.,
# you can pass additional parameters to the chat method.
# For a complete list of options, check the Ollama API documentation at:
# https://github.com/ollama/ollama/blob/main/docs/modelfile.md#valid-parameters-and-values
response = client.chat(
    model=MODEL_NAME,
    messages=[
        {"role": "user", "content": "What is the capital of France?"},
        {"role": "assistant", "content": "The capital of France is Paris."},
        {"role": "user", "content": "And who many people live there?"},
    ],
    options={
        "temperature": 0.7,
        "num_predict": 1024,
    }
)
print(response.message.content)

As of 2023, the population of Paris is approximately **2.1 million** people within the city limits. However, the greater Paris metropolitan area (which includes surrounding suburbs) has a population of over **11 million**! 

Would you like to know anything more about Paris?


In [8]:
# To force the model to generate a structured response, i.e. a JSON object,
# you can define a schema for the response and pass it. The schema can be defined using 
# Pydantic models and the built-in python types (more complex types are also supported, 
# check the pydantic documentation for more details).

from pydantic import BaseModel
class PopulationResponse(BaseModel):
    city: str
    population: int

response = client.chat(
    model=MODEL_NAME,
    messages=[
        {"role": "user", "content": "What is the capital of France?"},
        {"role": "assistant", "content": "The capital of France is Paris."},
        {"role": "user", "content": "And who many people live there?"},
    ],
    format=PopulationResponse.model_json_schema()
)
structured_response = PopulationResponse.model_validate_json(response.message.content)
print(structured_response)

city='Paris' population=2141636


In [14]:
import json

review_file = 'all_reviews.json'

with open(review_file, 'r', encoding='utf-8') as file:
    reviews = json.load(file)

for review in reviews:
    response = client.chat(
        model=MODEL_NAME,
        messages=[
            {"role": "user", 
             "content": 
                f"This is a peer review of a scientific article."
                f"Identify all valuable scientific questions raised by the reviewer."
                f"Only include questions that seek to clarify the scientific content of the article."
                f"Do not include questions about grammar, spelling or writing style."
                f"List the questions in a numbered list."
                f"Here is the review: {review}"}
        ],
        options={
            "temperature": 0.7,
            "num_predict": 1024,
        }
    )
    print(f"Review: {review} \n\n")
    print(f"Question: {response.message.content}\n")

Review: Please see the attached review. I think this manuscript has a lot of promise. See my comments for thoughts on how certain
potential issues could be addressed and how the addition of specific points of information could be of use.

Title
I do not see a reason why the word “future” should be in quotations. It’s meaning does not seem different because of it, so I
would remove the quotations from the title.

Abstract
Starting at line 6, the abstract becomes hard to understand. That sentence is grammatically incorrect, then o the following
sentence on line 9 the combination of more and less likely in the same sentence is confusing. Please split these into
separate sentences.

Introduction
1. I like the framing used in the first paragraph of the Introduction, but I also want to highlight that climate change and other
issues are not entirely zero-sum. There are certainly instances where an action taken for the future could also be beneficial
for the present. For instance, preserving a